# ⚠️ RESTART KERNEL

**Important:** The C library has been updated. Please restart the kernel before running cells below.

In [ ]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from numpy import pi, sqrt

# Matplotlib configuration
plt.style.use(['science', 'grid'])
plt.rcParams.update({'font.size': 18, 'text.usetex': True})

In [ ]:
# Load CVDV library (reload if already loaded)
import sys
import importlib
if 'interface' in sys.modules:
    importlib.reload(sys.modules['interface'])
from interface import CVDV

---
# Part 1: Single Register Tests

Test operations on a single register with large dimension.

In [ ]:
# Simulation parameters for single register tests
CV_LEVEL = 12      # 2^12 = 4096 grid points
WIGNER_N = 201     # Wigner function grid size
WIGNER_MAX = 5.0   # Wigner function range
PLOT_X_RANGE = 6   # x-axis range for plots

## Test 1: Vacuum State

In [ ]:
# Initialize single register system
# Register with 2^12 = 4096 points, dx automatically calculated
sim = CVDV([CV_LEVEL])

# Initialize to vacuum state
sim.setFock(0, 0)
sim.initStateVector()  # Build tensor product and upload to device

# Print system info
sim.info()

def debugPlot():
    wigner_complex = sim.getWignerSingleSlice(0, [ -1 ], wignerN=WIGNER_N, wXMax=WIGNER_MAX, wPMax=WIGNER_MAX)

    fig, ax = plt.subplots(figsize=(7, 6))
    vmax = np.max(np.abs(wigner_complex))
    im = ax.imshow(wigner_complex, extent=[-WIGNER_MAX, WIGNER_MAX, -WIGNER_MAX, WIGNER_MAX],
                origin='lower', cmap='RdBu', vmin=-vmax, vmax=vmax, aspect='equal')
    ax.set_xlabel(r'$q$')
    ax.set_ylabel(r'$p$')
    plt.colorbar(im, ax=ax)
    plt.tight_layout()
    plt.show()
    
debugPlot()

## Test 2: Displacement Operator

Test D(α) with different displacement parameters.

In [ ]:
# Test with real displacement
sim.setFock(0, 0)  # Reset to vacuum
sim.displacement(0, 2)
debugPlot()

In [ ]:
# Test with complex displacement
sim.displacement(0, -2 + 1.0j)
debugPlot()

In [ ]:
sim.ftQ2P(0)  # Fourier transform to momentum basis
debugPlot()

---
# Part 2: Multi-Register Tests

Test conditional displacement with 2 registers of different dimensions.

In [ ]:
# Free the simulator resources and reload library
if ' sim ' in globals():
    del globals()[' sim ']
    
DV_QUBITS = 1
CV_LEVEL_HYBRID = 10

sim = CVDV([DV_QUBITS, CV_LEVEL_HYBRID])

# Initialize: |0⟩ ⊗ |vac⟩
sim.setZero(0)  # Register 0 to |0⟩
sim.setFock(1, 0)  # Register 1 to vacuum
sim.initStateVector()  # Build tensor product and upload to device

sim.info()
def debugPlot():
    # Compute Wigner functions for each qubit basis state
    # For register 1 when register 0 is in |0⟩ or |1⟩
    wigner_0 = sim.getWignerSingleSlice(1, [0, -1], wignerN=WIGNER_N, 
                                        wXMax=WIGNER_MAX, wPMax=WIGNER_MAX)
    wigner_1 = sim.getWignerSingleSlice(1, [1, -1], wignerN=WIGNER_N,
                                        wXMax=WIGNER_MAX, wPMax=WIGNER_MAX)

    # Verify probabilities using new API
    probs = sim.measure(0)
    print(f"Register 0 probabilities: {probs}")
    print(f"P(qubit=|0⟩) = {probs[0]:.6f}")
    print(f"P(qubit=|1⟩) = {probs[1]:.6f}")
    print(f"Sum = {probs.sum():.6f} (should be 1.0)")

    # Plot both Wigner functions
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

    # |0⟩ component (displaced by +α)
    vmax = np.max(np.abs(wigner_0))
    im1 = ax1.imshow(wigner_0, extent=[-WIGNER_MAX, WIGNER_MAX, -WIGNER_MAX, WIGNER_MAX],
                    origin='lower', cmap='RdBu', vmin=-vmax, vmax=vmax, aspect='equal')
    ax1.set_xlabel(r'$q$')
    ax1.set_ylabel(r'$p$')
    ax1.set_title(r'$|0\rangle \otimes ?$')
    plt.colorbar(im1, ax=ax1)

    # |1⟩ component (displaced by -α)
    vmax = np.max(np.abs(wigner_1))
    im2 = ax2.imshow(wigner_1, extent=[-WIGNER_MAX, WIGNER_MAX, -WIGNER_MAX, WIGNER_MAX],
                    origin='lower', cmap='RdBu', vmin=-vmax, vmax=vmax, aspect='equal')
    ax2.set_xlabel(r'$q$')
    ax2.set_ylabel(r'$p$')
    ax2.set_title(r'$|1\rangle \otimes ?$')
    plt.colorbar(im2, ax=ax2)

    plt.tight_layout()
    plt.show()


## Test: Conditional Displacement CD(α)

Prepare |+⟩ ⊗ |vac⟩ and apply CD(α):
- CD(α)|+⟩|vac⟩ = (|0⟩|+α⟩ + |1⟩|-α⟩)/√2

In [ ]:
# Prepare |+⟩ = (|0⟩ + |1⟩)/√2
sim.hadamard(0, 0)  # Hadamard on qubit 0 of register 0
debugPlot()

In [ ]:
# Apply conditional displacement
alpha_cd = 2.0j
sim.cd(targetReg=1, ctrlReg=0, ctrlQubit=0, alpha=alpha_cd)
debugPlot()

In [ ]:
# Apply Hadamard again to complete entangling operation
sim.hadamard(0, 0)
debugPlot()